In [ ]:
import numpy as np
import math
import csv
import pandas as pd

In [ ]:
GEOMEAN_PER_PROBLEM = False
problem_names = [
    '1dconv_cpu',
    '1dconv_gpu',
    'dwspconv2d_gpu',
    'gemm_gpu'
]
leaderboard_files = ['leaderboards_2024-04-30 12:48:07.680398', 'leaderboards_2024-04-30 13:02:11.682227', 'leaderboards_2024-04-30 13:10:16.072927', 'leaderboards_2024-04-30 13:18:19.082314', 'leaderboards_2024-04-30 13:34:04.054430']


In [ ]:
def add_problem_times_to_map(orig_map, problem_name, problem_array):
    for item in problem_array:
        id, time = item[0], item[1]
        if id not in orig_map:
            orig_map[id] = {}
        if problem_name not in orig_map[id]:
            orig_map[id][problem_name] = []
        orig_map[id][problem_name].append(float(time))
    return orig_map



In [ ]:
results = {}
for file in leaderboard_files:
    leaderboard = np.load(file + '.npy', allow_pickle=True)
    for i in range(0, len(problem_names)):
        add_problem_times_to_map(results, problem_names[i], leaderboard[i])

In [ ]:
results

In [ ]:
# Merge results arrays into one number using geometric mean
problem_results = {}
for id in results:
    problem_results[id] = {}
    for problem in results[id]:
        times = results[id][problem]
        problem_results[id][problem] = math.prod(times)**(1/len(times))

problem_results

In [ ]:
final_results = {}

# Merge results of all problems into 1 number using mean
for id in problem_results:
    times = problem_results[id].values()
    if GEOMEAN_PER_PROBLEM:
        final_results[id] = math.prod(times)**(1/len(times))
    else:
        final_results[id] = sum(times) / len(times)


final_results

In [ ]:
# Sort final_results by value ascendingly
final_results = dict(sorted(final_results.items(), key=lambda item: item[1]))

final_results

In [ ]:
# Save final_results to a csv

with open('final_results.csv', mode='w') as file:
    writer = csv.writer(file)
    cols = ['id']
    cols+= problem_names
    cols.append('geomean')
    writer.writerow(cols)
    for id in final_results:
        row = [id]
        for problem in problem_names:
            row.append(problem_results[id][problem])
        row.append(final_results[id])
        writer.writerow(row)


In [ ]:
repos = np.load('repos.npy', allow_pickle=True)
repos.item()

In [ ]:
github_handles = pd.read_csv('github_handles.csv')

github_handles

In [ ]:
id_to_netid = {}
for index, row in github_handles.iterrows():
    github_username = row['github_username']
    anon_id = None
    for id, vals in repos.item().items():
        if vals[0] == f'a3-{github_username}':
            anon_id = id
            break
    id_to_netid[anon_id] = row['identifier']

id_to_netid

In [ ]:
# Write CSV that contains NetID along with rest of results
with open('final_results_with_netid.csv', mode='w') as file:
    writer = csv.writer(file)
    cols = ['id', 'netid']
    cols+= problem_names
    cols.append('geomean')
    writer.writerow(cols)
    for id in final_results:
        row = [id, id_to_netid[id]]
        for problem in problem_names:
            row.append(problem_results[id][problem])
        row.append(final_results[id])
        writer.writerow(row)